In [1]:
import os
from itertools import product

import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, atpe, fmin, hp, tpe
import sys
sys.path.append('../')
from core.generator import Main as Model

In [2]:
import igraph
igraph.__version__

'0.10.4'

In [2]:
num_of_nodes=1000
min_degree = [1,5,7,9]
max_degree = [300,500,700,900]
mu = [0.1,0.5,0.9]
tau1 = [2.0,2.2,2.4,2.6]
sigma_init_list = list(np.arange(0.8,1.31,0.05)) 
for tau11 in tau1:
    for L in [5,10,15,20]:
        for mu1 in mu:
            for min_d in min_degree:
                for max_d in max_degree:
                    for sigma in sigma_init_list:
                        for eta in [0.05,0.2,0.4,0.6,0.8,1,3,5]:
                            for rho in [0.6,0.7,0.9,0.95]:
                                model = Model(num_of_nodes=1000, max_d=max_d,
                                                     L=L, 
                                                     etta=eta,ro=rho,
                                                     mu=mu1, sigma_init=sigma,
                                                     sigma_every=1, 
                                                     d=128,
                                                     power=tau11,
                                                     min_d=min_d)

                                G,_ = model.making_graph()
                                print(model.statistics())

{'Power': 2.0, 'N': 1000, 'M': 300, 'L': 5, 'Eta': 0.05, 'Ro': 0.6, 'Mu': 0.1, 'Disper': 0.8, 'd': 128, 'Avg Degree': 3.802, 'Cluster': 0.07688363553552471, 'Density': 0.003805805805805806, 'Min degree': 1, 'Feature Assort': 0.06156626126053758, 'Label Assort': 0.2407997186347568, 'connected components': 2, 'Avg shortest path': 2.148123729907156}
{'Power': 2.0, 'N': 1000, 'M': 300, 'L': 5, 'Eta': 0.05, 'Ro': 0.7, 'Mu': 0.1, 'Disper': 0.8, 'd': 128, 'Avg Degree': 4.038, 'Cluster': 0.0982497465449651, 'Density': 0.004042042042042042, 'Min degree': 1, 'Feature Assort': 0.05145283366969849, 'Label Assort': 0.26336135959563994, 'connected components': 1, 'Avg shortest path': 3.3555275275275274}
{'Power': 2.0, 'N': 1000, 'M': 300, 'L': 5, 'Eta': 0.05, 'Ro': 0.9, 'Mu': 0.1, 'Disper': 0.8, 'd': 128, 'Avg Degree': 3.93, 'Cluster': 0.15390204838601343, 'Density': 0.003933933933933934, 'Min degree': 1, 'Feature Assort': 0.04330676427449949, 'Label Assort': 0.25472762589528797, 'connected componen

KeyboardInterrupt: 

In [2]:
# Setting up hyperparameters space
hp_space = {'sigma_init': hp.uniform('sigma_init', 0.8,1.1), 
            'power': hp.uniform('power', 2, 3),
           'max_d': hp.quniform('max_d',300,1000,100),'L':hp.quniform('L',5,20,5),'etta':hp.uniform('etta',0,5),
            'ro':hp.uniform('ro',0,1),
            'mu':hp.uniform('mu',0.1,0.95),
            'min_d':hp.quniform('min_d',1,10,1)}

In [4]:
df_bench=pd.read_csv('data_benchmark/bench_properties.csv')
df_bench=df_bench.drop(columns=['Unnamed: 0'])

In [7]:
df_bench_left = pd.DataFrame(columns='Label assort','Feature assort','Cluster','Avg shortest path','Avg Degree','connected components')

,Label assort,Feature assort,Cluster,Avg shortest path,Avg Degree,connected components
0,0.393022,0.069726,0.010032,2.125705,2.478,6.0
1,0.393022,0.069726,0.010032,2.125705,2.478,6.0
2,0.393022,0.069726,0.010032,2.125705,2.478,6.0
3,0.278693,0.104036,0.010060,2.828765,2.296,3.0
4,0.278693,0.104036,0.010060,2.828765,2.296,3.0
5,0.271792,0.072670,0.009787,2.068783,2.228,9.0
6,0.975543,0.300161,0.208536,4.158685,4.854,1.0
7,0.975543,0.300161,0.208536,4.158685,4.854,1.0
8,0.975543,0.300161,0.208536,4.158685,4.854,1.0
9,0.983617,0.421413,0.201031,4.119399,4.790,1.0


In [9]:
# OUT_PAR_NAMES = ['label assort','feature assort', 'cluster','avg shortest path','average degree']
OUT_PAR_NAMES = ['Label Assort','Feature Assort', 'Cluster',
                 'Avg shortest path','Avg Degree','connected components']

Npar_out = len(OUT_PAR_NAMES)

def par_to_array(par):
    rez = []
    for p in OUT_PAR_NAMES:
        if p not in par.keys():
            return None
        rez.append(par[p])
    rez = np.array(rez)    
    return rez
        
def array_to_par(arr):    
    return {OUT_PAR_NAMES[i]:arr[i] for i in range(len(OUT_PAR_NAMES))}

def loss_func(pred, target):
    weight = np.ones(Npar_out)
    weight[-1] = 0.3
    
    return (((  (pred-target)/target   )**2) * weight).sum()

class Objective:
    def __init__(self, target):
        self.target = target
    
    def __call__(self, args):
        model = Main(N=1000, max_d=int(args['max_d']), 
                     L=int(args['L']), etta=args['etta'],ro=args['ro'],
                     mu=args['mu'], sigma_init=args['sigma_init'],
                     sigma_every=1, d=128, power=args['power'], min_d=int(args['min_d']))
        G,_ = model.making_graph()
        
        stats = model.statistics()
        out_pars = par_to_array(stats)
        loss = loss_func(out_pars, self.target)  
        
        np.stack((np.ones(10), np.ones(10)), axis=0)
        
#         print((out_pars-self.target)/self.target, self.target)
        
        print(pd.DataFrame(np.vstack((out_pars, self.target)), 
                           index = ['pred', 'true'], columns=OUT_PAR_NAMES))
        print('loss  ', loss)
        print('-----------------------------------------')
        resp = array_to_par(out_pars)
        resp['loss'] = loss.sum()
        resp['status'] = STATUS_OK
        return resp
    
    def early_stop(self, trials):
        out_pars = par_to_array(trials.trials[-1]['result'])
        limits = {'Label Assort': 0.05,'Feature Assort': 0.05, 'Cluster': 0.05,
                     'Avg shortest path': 0.5,'Avg Degree':1.5,'connected components': 10}
        
        diff = np.abs((out_pars-self.target)/self.target)
        return np.all(diff < par_to_array(limits) ), []

In [4]:
target_parameters = np.array([x for x in product(
                    [0.1,0.5,0.9], 
                    [0.1,0.5,0.9], 
                    [0.01,0.2,0.5], 
                    [3,5,7], 
                    [5,20,40],
                    [1])])

In [7]:
target_parameters=[]
for l_a_trgt in [0.1,0.5,0.9]:
    for f_a_trgt in [0.6,0.88,0.92]:
        for cl_trgt in [0.1,0.25,0.5]:
            for asp_trgt in [3,2,1,4.5,5,5.5,6,6.5,7]:
                for a_deg_trgt in [2,5,20,40]:
                    target_parameters.append([l_a_trgt,f_a_trgt,cl_trgt,asp_trgt,a_deg_trgt,1])
                        

In [4]:
len(target_parameters)

3564

In [37]:
m = 500
trials = Trials()

for  targets in (target_parameters):  
    targets_dict = array_to_par(targets)
    for tr in trials.trials:
        par = par_to_array(tr['result'])
        if par is not None:
            tr['result']['loss'] = loss_func(par, targets)        
    
    max_eval = len(trials.trials)+m
    #best = fmin(objective, hp_space,trials=trials, algo=tpe.suggest,max_evals=max_eval,early_stop_fn=early_stop)
    
    objective = Objective(targets)
    
    best = fmin(objective, hp_space, trials=trials,
                algo=atpe.suggest,max_evals=max_eval,
                early_stop_fn=objective.early_stop)
    
    #G = trials.best_trial['result']['graph'] 
    
    print('id of the best trial',trials.best_trial['tid'] )
        

        

      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.527581        0.285956  0.175486           2.452184        24.5   
true      0.100000        0.100000  0.100000           2.000000         5.0   

      connected components  
pred                   1.0  
true                   1.0  
loss                                                                                                                   
37.57146539609194                                                                                                      
-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.557472        0.197843  0.218757            2.39418      24.218   
true      0.100000        0.100000  0.100000            2.00000       5.000

loss                                                                                                                   
23.49906207432396                                                                                                      
-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.725952        0.112207  0.263463           2.668759      19.424   
true      0.100000        0.100000  0.100000           2.000000       5.000   

      connected components  
pred                   1.0  
true                   1.0  
loss                                                                                                                   
50.30243421104619                                                                                                      
----------------------------------

-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.146964        0.194323  0.085783                0.0       3.876   
true      0.100000        0.100000  0.100000                2.0       5.000   

      connected components  
pred                   2.0  
true                   1.0  
loss                                                                                                                   
2.4809939475738036                                                                                                     
-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.230221        0.117395

loss                                                                                                                   
2.0123485906068503                                                                                                     
-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.150034        0.115874  0.070278                0.0        4.04   
true      0.100000        0.100000  0.100000                2.0        5.00   

      connected components  
pred                   3.0  
true                   1.0  
loss                                                                                                                   
2.600747807046265                                                                                                      
----------------------------------

-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.215817        0.128389  0.108982                0.0       3.566   
true      0.100000        0.100000  0.100000                2.0       5.000   

      connected components  
pred                   5.0  
true                   1.0  
loss                                                                                                                   
7.312265194074564                                                                                                      
-----------------------------------------                                                                              
      Label Assort  Feature Assort  Cluster  Avg shortest path  Avg Degree  \                                          
pred      0.135702         0.16261

...

-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.156284        0.087503  0.107203                0.0       3.842   
true      0.100000        0.100000  0.100000                3.0       5.000   

      connected components  
pred                   3.0  
true                   1.0  
loss                                                                                                                   
2.591237564730065                                                                                                      
-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.189992        0.106667

loss                                                                                                                   
3.6044104428839012                                                                                                     
-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.158421         0.09279  0.090607                0.0       3.534   
true      0.100000         0.10000  0.100000                3.0       5.000   

      connected components  
pred                   9.0  
true                   1.0  
loss                                                                                                                   
20.641293062549543                                                                                                     
----------------------------------

-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.283951        0.076544  0.097836                0.0       3.532   
true      0.100000        0.100000  0.100000                3.0       5.000   

      connected components  
pred                   3.0  
true                   1.0  
loss                                                                                                                   
5.725466386702397                                                                                                      
-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.182873        0.091256

loss                                                                                                                   
1.6631853605463944                                                                                                     
-----------------------------------------                                                                              
      Label Assort  Feature Assort   Cluster  Avg shortest path  Avg Degree  \                                         
pred      0.202389        0.649649  0.080373           2.077314       3.184   
true      0.100000        0.100000  0.100000           3.000000       5.000   

      connected components  
pred                   3.0  
true                   1.0  
loss                                                                                                                   
32.72479226173037                                                                                                      
----------------------------------

KeyboardInterrupt: 

In [8]:
class Main():
    def __init__(self,m,parameters_check):
        super(Main, self).__init__()
        self.hp_space = {'sigma_init': hp.uniform('sigma_init', 0.8,1.1), 
            'power': hp.uniform('power', 2, 3),
            'max_d': hp.quniform('max_d',300,1000,100),'L':hp.quniform('L',5,20,5),'etta':hp.uniform('etta',0,5),
            'ro':hp.uniform('ro',0,1),
            'mu':hp.uniform('mu',0.1,0.95),
            'min_d':hp.quniform('min_d',1,10,1)} 
        self.OUT_PAR_NAMES = ['Label Assort','Feature Assort', 'Cluster',
                 'Avg shortest path','Avg Degree','connected components']
        self.m=m
        self.parameters= parameters_check
        self.parameters_check=parameters_check
        
       # for l_a_trgt in [0.1,0.5,0.9]:
        #    for f_a_trgt in [0.1,0.5,0.9]:
         #       for cl_trgt in [0.01,0.1,0.2,0.3,0.5]:
          #          for asp_trgt in [2,3,4,5,6,7]:
           #             for a_deg_trgt in [2,5,10,15,20,25,30,35,40]:
                            
            #                if (asp_trgt <= 3 and a_deg_trgt<=10) or (asp_trgt > 3 and a_deg_trgt>20):
             #                   pass
              #              else:
               #                 self.parameters.append([l_a_trgt,f_a_trgt,cl_trgt,asp_trgt,a_deg_trgt,1])
                #                self.parameters_check.append([l_a_trgt,f_a_trgt,cl_trgt,asp_trgt,a_deg_trgt,1])

                                
        self.Npar_out = len(self.OUT_PAR_NAMES)
        self.trials=Trials()
        self.max_eval=0
        self.limits = {'Label Assort': 0.3,'Feature Assort': 0.3, 'Cluster': 0.05,
                     'Avg shortest path': 0.3,'Avg Degree':1.5,'connected components': 10}
        self.df_bench = pd.DataFrame(columns=['Label assort','Feature assort','Cluster','Avg shortest path','Avg Degree','connected components'])

    def par_to_array(self,par):
        rez = []
        for p in self.OUT_PAR_NAMES:
            if p not in par.keys():
                return None
            rez.append(par[p])
        rez = np.array(rez)    
        return rez

    def array_to_par(self,arr):    
        return {self.OUT_PAR_NAMES[i]:arr[i] for i in range(len(self.OUT_PAR_NAMES))}

    def loss_func(self,pred, target):
        weight = np.ones(self.Npar_out)
        weight[-1] = 0.3

        return (abs((  (pred-target)/target   )) * weight).sum()
    
    def early_stop(self, trials):
        out_pars = self.par_to_array(trials.trials[-1]['result'])
        
        for num, targets_check in enumerate(self.parameters_check):
            
            diff = np.abs((out_pars - targets_check)/targets_check)
            if np.all(diff < self.par_to_array(self.limits) ):
                return True,[]
        return False, []
    
    def objective(self, args):
        model = Model(N=1000, max_d=int(args['max_d']), 
                     L=int(args['L']), 
                     etta=args['etta'],ro=args['ro'],
                     mu=args['mu'], sigma_init=args['sigma_init'],
                     sigma_every=1, 
                     d=128,
                     power=args['power'],
                     min_d=int(args['min_d']))
        
        G,_ = model.making_graph()
        
        stats = model.statistics()
        out_pars = self.par_to_array(stats)
        loss = self.loss_func(out_pars, self.targets)  
        nums_to_del = []
        for num, targets_check in enumerate(self.parameters_check):
                    name =  "".join(list(map(lambda x:str(x),  targets_check[:-1])))# str(l_a_trgt_t)+str(f_a_trgt_t)+str(cl_trgt_t)+str(asp_trgt_t)+ str(a_deg_trgt_t)
                    if os.path.exists('data_benchmark_left/graph_'+str(name)+'.pickle'):
                        nums_to_del.append(num)
                    else:
                        diff = np.abs((out_pars - targets_check)/targets_check)
                        if np.all(diff < self.par_to_array(self.limits) ) :
                            to_append =  list(out_pars)
                            #print(to_append)
                            row_series = pd.Series(to_append, index = self.df_bench.columns)
                            self.df_bench = self.df_bench.append(row_series, ignore_index = True)
                            #self.df_bench.to_csv('data_benchmark_4/bench_properties.csv')    

                            node_attr = []
                            for i,attr in G.nodes('attribute'):
                                node_attr.append(attr.tolist())
                            labels = []
                            for i,lab in G.nodes('label'):
                                labels.append(lab)
                            labels=np.array(labels)

                            name =  "".join(list(map(lambda x:str(x),  targets_check[:-1])))# str(l_a_trgt_t)+str(f_a_trgt_t)+str(cl_trgt_t)+str(asp_trgt_t)+ str(a_deg_trgt_t)
                            np.save('data_benchmark_left/graph_'+str(name)+'_edgelist.npy',np.array(G.edges()))
                            np.save('data_benchmark_left/graph_'+str(name)+'_attr.npy',node_attr)
                            with open('data_benchmark_left/graph_'+str(name)+'.pickle','wb') as f:
                                pickle.dump(G,f)
                            np.save('data_benchmark_left/graph_'+str(name)+'_labels.npy',labels)
                            nums_to_del.append(num)
                            
        for index in sorted(nums_to_del, reverse=True):
            del self.parameters_check[index]        

        #print(pd.DataFrame(np.vstack((out_pars, self.targets)), 
                         #  index = ['pred', 'true'], columns=self.OUT_PAR_NAMES))
        #print('loss  ', loss)
        #print('-----------------------------------------')
        resp = self.array_to_par(out_pars)
        resp['loss'] = loss.sum()
        resp['status'] = STATUS_OK
        return resp
    
    def run(self):
        
        for targets in (self.parameters):
            targets_dict = self.array_to_par(targets)
            self.targets=targets
            
            for tr in self.trials.trials:
                par = self.par_to_array(tr['result'])
                if par is not None:
                    tr['result']['loss'] = self.loss_func(par, targets)

            self.max_eval = len(self.trials.trials)+self.m
            best = fmin(self.objective, 
                            self.hp_space, 
                            trials=self.trials,
                            algo=atpe.suggest,
                            max_evals=self.max_eval,
                            early_stop_fn=self.early_stop)

In [9]:
main=Main(m=500,parameters_check=target_parameters)
main.run()

100%|████████████████████████████████████████▊| 38351/38500 [3:01:20<00:42,  3.52trial/s, best loss: 1.142278508855859]


KeyboardInterrupt: 

In [9]:
import pickle
with open('parameters_to_check.pickle','wb') as f:
    pickle.dump(main.parameters_check,f)

In [7]:
import pickle
with open('parameters_to_check.pickle','rb') as f:
    parameters_check=pickle.load(f)